In [1]:
# Import the required libraries and dependencies
import pandas as pd
import datetime as dt
from prophet import Prophet
from clean_data import clean_data


In [2]:
# Create path to CSV file
csv_path = "Metro_Interstate_Traffic_Volume.csv"
# Import the CSV into a pandas DataFrame
df = pd.read_csv(csv_path)
# Run imported DataFrame through clean_data function
clean_df = clean_data(df)

In [ ]:
sorted_df = clean_df.sort_values(by='date_time', ascending=True)
sorted_df.head(60)


In [ ]:
def identify_gaps(df, output_file):
    # Define the expected range of hourly datetimes
    start_date = pd.to_datetime('2012-10-02 09:00:00')
    end_date = pd.to_datetime('2018-09-30 23:00:00')
    expected_dates = pd.date_range(start=start_date, end=end_date, freq='H')

    # Extract the actual datetimes from the DataFrame
    actual_dates = df.index

    # Identify missing datetimes
    missing_dates = expected_dates[~expected_dates.isin(actual_dates)]

    # Export missing datetimes to a CSV file
    missing_dates.to_frame(name='missing_datetime').to_csv(output_file, index=False)

# Assuming 'df' is your DataFrame
output_file = 'missing_datetimes.csv'
identify_gaps(clean_df, output_file)

In [ ]:
def identify_gaps(df, output_missing_file, output_long_gaps_file):
    # Define the expected range of hourly datetimes with frequency 'H'
    start_date = pd.to_datetime('2012-10-02 09:00:00')
    end_date = pd.to_datetime('2018-09-30 23:00:00')
    expected_dates = pd.date_range(start=start_date, end=end_date, freq='H')

    # Extract the actual datetimes from the DataFrame
    actual_dates = df.index

    # Identify missing datetimes
    missing_dates = expected_dates[~expected_dates.isin(actual_dates)]

    # Identify gaps longer than 6 hours
    gap_lengths = (missing_dates - missing_dates.shift(1)).fillna(pd.Timedelta(0))
    long_gaps = missing_dates[gap_lengths >= pd.Timedelta(hours=6)]

    # Export missing datetimes to a CSV file
    missing_dates.to_frame(name='missing_datetime').to_csv(output_missing_file, index=False)

    # Export long gaps to a separate CSV file
    long_gaps.to_frame(name='long_gap_datetime').to_csv(output_long_gaps_file, index=False)

# Assuming 'df' is your DataFrame
output_missing_file = 'missing_datetimes.csv'
output_long_gaps_file = 'long_gaps.csv'
identify_gaps(clean_df, output_missing_file, output_long_gaps_file)

In [ ]:
len(clean_df)

In [ ]:
holiday_rows = df['is_holiday']

In [4]:
print(df['holiday'].unique())

[nan 'Columbus Day' 'Veterans Day' 'Thanksgiving Day' 'Christmas Day'
 'New Years Day' 'Washingtons Birthday' 'Memorial Day' 'Independence Day'
 'State Fair' 'Labor Day' 'Martin Luther King Jr Day']
